# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherdf = pd.read_csv('../WeatherPy/city_weather.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
#gmaps.configure(config=gkey)

TypeError: configure() got an unexpected keyword argument 'config'

In [4]:
#Setting values
locations = weatherdf[["Lat", "Lng"]]
humidity = weatherdf["Humidity"]

#creating layout for map to be easier to read
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#using gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

#the heatlayer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                 max_intensity = 100,point_radius = 2.5)
#adding the layer
fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
perfect_df = weatherdf[(weatherdf['Temperature'] >= 70) & (weatherdf['Temperature'] <= 80)]
perfect_df = perfect_df[perfect_df['Wind Speed'] <= 10]
perfect_df = perfect_df[perfect_df['Clouds'] <= 0]
hotel_df = perfect_df
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
22,86049,Jalu,LY,29.0331,21.5482,75.72,16,0,7.20
37,3833367,Ushuaia,AR,-54.8000,-68.3000,75.20,38,0,5.75
55,286621,Salalah,OM,17.0151,54.0924,77.00,47,0,4.61
130,3443061,Chui,UY,-33.6971,-53.4616,76.32,72,0,8.41
134,1312609,Magway,MM,20.1500,94.9167,73.20,28,0,3.94
158,1278017,Bādāmi,IN,15.9150,75.6768,76.37,20,0,7.85
215,1292037,Thayetmyo,MM,19.3167,95.1833,71.06,30,0,2.82
233,107304,Buraidah,SA,26.3260,43.9750,71.60,43,0,2.30
287,108410,Riyadh,SA,24.6877,46.7219,71.60,15,0,8.05
291,3598787,Champerico,GT,14.3000,-91.9167,75.63,74,0,7.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ""
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
22,86049,Jalu,LY,29.0331,21.5482,75.72,16,0,7.20,
37,3833367,Ushuaia,AR,-54.8000,-68.3000,75.20,38,0,5.75,
55,286621,Salalah,OM,17.0151,54.0924,77.00,47,0,4.61,
130,3443061,Chui,UY,-33.6971,-53.4616,76.32,72,0,8.41,
134,1312609,Magway,MM,20.1500,94.9167,73.20,28,0,3.94,
158,1278017,Bādāmi,IN,15.9150,75.6768,76.37,20,0,7.85,
215,1292037,Thayetmyo,MM,19.3167,95.1833,71.06,30,0,2.82,
233,107304,Buraidah,SA,26.3260,43.9750,71.60,43,0,2.30,
287,108410,Riyadh,SA,24.6877,46.7219,71.60,15,0,8.05,
291,3598787,Champerico,GT,14.3000,-91.9167,75.63,74,0,7.36,


In [7]:
for index, row in hotel_df.iterrows():
   #So not to burn up API
    time.sleep(1) 
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": gkey,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
    
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
22,86049,Jalu,LY,29.0331,21.5482,75.72,16,0,7.20,Jalu Hotel
37,3833367,Ushuaia,AR,-54.8000,-68.3000,75.20,38,0,5.75,Wyndham Garden Ushuaia Hotel del Glaciar
55,286621,Salalah,OM,17.0151,54.0924,77.00,47,0,4.61,HYATT SALALAH HOTEL
130,3443061,Chui,UY,-33.6971,-53.4616,76.32,72,0,8.41,Bertelli Chuí Hotel
134,1312609,Magway,MM,20.1500,94.9167,73.20,28,0,3.94,Man Thi Tar Hotel Minbu
158,1278017,Bādāmi,IN,15.9150,75.6768,76.37,20,0,7.85,HOTEL BADAMI COURT
215,1292037,Thayetmyo,MM,19.3167,95.1833,71.06,30,0,2.82,Ngwe Sin Kyal Guest House unit 2
233,107304,Buraidah,SA,26.3260,43.9750,71.60,43,0,2.30,فندق تاورز HOTELTOWERS
287,108410,Riyadh,SA,24.6877,46.7219,71.60,15,0,8.05,voco Riyadh
291,3598787,Champerico,GT,14.3000,-91.9167,75.63,74,0,7.36,Hotel Posada del Mar


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…